In [1]:
!pip install opensearch-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [opensearch-py]


In [1]:
import os
# from elasticsearch import Elasticsearch
import numpy as np
import pandas as pd
import subprocess
from subprocess import Popen, PIPE, STDOUT
from opensearchpy import OpenSearch


https://www.elastic.co/docs/deploy-manage/deploy/self-managed/install-elasticsearch-from-archive-on-linux-macos

In [4]:
conda install wget

Jupyter detected...
2 channel Terms of Service accepted
Channels:
 - defaults
Platform: osx-arm64
Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - wget


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    wget-1.25.0                |       hbc1a67d_2         747 KB
    ------------------------------------------------------------
                                           Total:         747 KB

The following NEW packages will be INSTALLED:

  wget               pkgs/main/osx-arm64::wget-1.25.0-hbc1a67d_2 



                                                                                
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [1]:
%%capture
# Download and extract elasticsearch
#!wget https://d3g5vo6xdbdb9a.cloudfront.net/tarball/opendistro-elasticsearch/opendistroforelasticsearch-1.13.2-linux-x64.tar.gz
#!tar -zxf opendistroforelasticsearch-1.13.2-linux-x64.tar.gz

In [2]:
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-9.2.1-darwin-x86_64.tar.gz
# curl https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-9.2.1-darwin-x86_64.tar.gz.sha512 | shasum -a 512 -c -
# tar -xzf elasticsearch-9.2.1-darwin-x86_64.tar.gz
# cd elasticsearch-9.2.1/

--2025-11-30 08:31:34--  https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-9.2.1-darwin-x86_64.tar.gz
Resolving artifacts.elastic.co (artifacts.elastic.co)... 34.120.127.130, 2600:1901:0:1d7::
Connecting to artifacts.elastic.co (artifacts.elastic.co)|34.120.127.130|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 470383892 (449M) [binary/octet-stream]
Saving to: ‘elasticsearch-9.2.1-darwin-x86_64.tar.gz.1’

elasticsearch-9.2.1 100%[===================>] 448.59M  8.02MB/s    in 68s     

2025-11-30 08:32:43 (6.58 MB/s) - ‘elasticsearch-9.2.1-darwin-x86_64.tar.gz.1’ saved [470383892/470383892]



In [5]:
# !chown -R daemon:daemon ./opendistroforelasticsearch-1.13.2
# !chown -R daemon:daemon ./elasticsearch-9.2.1

In [ ]:
server = Popen(['./opendistroforelasticsearch-1.13.2/opendistro-tar-install.sh'], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1))
!sleep 30

In [ ]:
%%bash
curl -X GET "https://localhost:9200" -u 'admin:admin' --insecure

In [ ]:
%%bash
curl -X GET "https://localhost:9200/_cat/plugins?v" -u 'admin:admin' --insecure

In [ ]:
host = 'localhost'
port = 9200
auth = ('admin', 'admin') 
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)

In [ ]:
index_name = 'test'
index_body = {
  'settings': {
    'index': {
      'number_of_shards': 4
    }
  }
}

In [ ]:
response = client.indices.create(index_name, body=index_body)

In [ ]:
print('\nCreating index:')
print(response)

In [ ]:
document = {
  'title': 'Moneyball',
  'director': 'Bennett Miller',
  'year': '2011'
}
id = '1'

response = client.index(
    index = index_name,
    body = document,
    id = id,
    refresh = True
)

In [ ]:
print('\nAdding document:')
print(response)

In [ ]:
q = 'miller'
query = {
  'size': 5,
  'query': {
    'multi_match': {
      'query': q,
      'fields': ['title^2', 'director']
    }
  }
}

response = client.search(
    body = query,
    index = index_name
)

In [ ]:
print('\nSearch results:')
print(response)

In [ ]:
# data = pd.read_csv("../input/treccovid-round-1/topic_set.csv")
# data.head()

In [ ]:
# model_name = "bert-base-nli-mean-tokens"
# model = SentenceTransformer(model_name)

In [ ]:
# data['batch'] = data['narrative'].apply(lambda t: model.encode([t])[0].tolist())

In [ ]:
# def prepare_es_data(index, df):
#     records = df.to_dict(orient="records")
#     es_data = []
#     for idx, record in enumerate(records):
#         meta_dict = {
#               "index": {
#                   "_index": index, 
#                   "_id": idx
#               }
#           }
#         es_data.append(meta_dict)
#         es_data.append(record)
#     return es_data

In [ ]:
# def index_es_data(index, es_data):
#     res = es_client.bulk(index=index, body=es_data, refresh = True)
#     print("Errors: {}, Num of records indexed: {}".format(res["errors"], len(res["items"])))

In [ ]:
# es_data = prepare_es_data(index="sentence",  df=data)

In [ ]:
# index_es_data(index="sentence", es_data=es_data)

In [ ]:
# vecquery = model.encode(["coronavirus social distancing impact"])[0]

In [ ]:

# query = {
#   "size": 6,
#   "query": {
#     "script_score": {
#       "query" : {
#         "match_all": {}
#       },
#       "script": {
#         "source": "1 / (1 + l2norm(params.queryVector, 'batch'))",
#         "params": {
#           "queryVector": vecquery
#         }
#       }
#     }
#   },
#   "sort": { 
#       "_score": { "order": "desc" } 
#   }
# }

# results = es_client.search(index="sentence", body=query)

In [ ]:
# for hit in results["hits"]["hits"]:
#     print([hit["_source"]["query"],"---", hit["_score"]])

In [ ]:
# 0.06816692 - 0.06667824